# Statistical Language Modeling with NLTK


### Recommended Reading

- Dan Jurafsky and James H. Martin's **Speech and Language Processing** ([3rd ed. draft](https://web.stanford.edu/~jurafsky/slp3/))


### Covered Material

- SLP
  - [Chapter 3: N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf)


<!-- ### Requirements

- [NLTK](https://www.nltk.org/) -->


## 1. Ngrams and Ngram Counting


[n-gram](https://en.wikipedia.org/wiki/N-gram) is a contiguous sequence of $N$ items from a given sequence of text or speech. An N-gram model can model sequences, notably natural languages, by using the statistical properties of N-grams (based on N-gram counts).

**Example**:

- character N-grams: mice
- word N-grams: the answer is 42

|                     | 1-gram                           | 2-gram                             | 3-gram                   |
| ------------------- | -------------------------------- | ---------------------------------- | ------------------------ |
|                     | unigram                          | bigram                             | trigram                  |
| _Markov Order_      | 0                                | 1                                  | 2                        |
| _Character N-grams_ | `['m', 'i', 'c', 'e']`           | `['mi', 'ic', 'ce']`               | `['mic', 'ice']`         |
| _Word N-grams_      | `['the', 'answer', 'is' , '42']` | `['the answer', 'answer is', ...]` | `['the answer is', ...]` |

You can imagine this as a sliding window with a width of $N$ and a stride to the right of 1 over tokens or characters.


### 1.1. Counting Ngrams

_Frequency List_ of a corpus is essentially a unigram count, i.e. the length of the sequences to count is 1. Indeed, N-gram count is just a generalization of _Frequency List_ in which the length of the sequences to count is given by $N$ instead of only 1 as default.

Thus, with N-gram count we can compute the count by taking sequences of 2 items (bigrams), 3 items (trigrams), etc.


#### 1.1.1. Preparing Data

The required data format for n-gram counting is a _list-of-lists_ (lists of sentences consisting of lists of words):

```
[
     ['the', 'answer', 'is', '42'],
     ['the', 'mice', 'said']
]
```


#### 1.1.2. Sentence beginning & end tags

> Including sentence boundary markers leads to a better model. To do that we need to augment each sentence with a special symbols for beginning and end of sentence tags (`<s>` and `</s>`, respectively). The beginning of the sentence (BOS) tag gives the bigram context of the first word; and encodes probability of a word to start a sentence. Adding the end of the sentence (EOS) tag delimits the end of a sentence, i.e. it represents the final state of the sequence. The EOS is also used to signaling when the generation of a new sequence can be halted.


> Moreover, for n-grams larger than 2, we’ll need to assume extra context for the contexts to the left and right of the sentence boundaries. For example, to compute trigram probabilities at the very beginning of the sentence, we can use two pseudo-words for the first trigram (i.e. `['<s>', '<s>', w1]`). Alternatively, we can use [back-off](https://en.wikipedia.org/wiki/Katz%27s_back-off_model), and use the `['<s>', w1]` bigram probability.


**Example**:
`['<s>', 'the', 'answer', 'is', '42', '</s>']`


#### 1.1.3. NLTK Utility Functions

NLTK provides utility functions for padding sequences.


In [28]:
sent = ["the", "answer", "is", "42"]

In [29]:
from nltk.util import pad_sequence

list(
    pad_sequence(
        sent,  # input sequence
        pad_left=True,
        left_pad_symbol="<s>",
        pad_right=True,
        right_pad_symbol="</s>",
        n=2,  # padding for bigrams
    )
)

['<s>', 'the', 'answer', 'is', '42', '</s>']

Another NLTK function wraps this utility function with default arguments to provide a more convenient interface.


In [30]:
from nltk.lm.preprocessing import pad_both_ends

list(pad_both_ends(sent, n=1))

['the', 'answer', 'is', '42']

#### 1.1.4. Extracting N-grams

NLTK provides a function to extract N-grams from a sequence, which also performs padding, if required arguments are provided.


In [31]:
from nltk.util import ngrams

list(ngrams(sent, 2))

[('the', 'answer'), ('answer', 'is'), ('is', '42')]

In [32]:
bigrams = ngrams(
    sent,
    2,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
)
list(bigrams)

[('<s>', 'the'),
 ('the', 'answer'),
 ('answer', 'is'),
 ('is', '42'),
 ('42', '</s>')]

Additionally, NLTK provides wrapper functions to extract bigrams and trigrams.


In [33]:
from nltk.util import bigrams, trigrams

print(list(bigrams(sent)))
print(list(trigrams(sent)))

print(list(bigrams(pad_both_ends(sent, n=2))))
print(
    list(
        bigrams(
            sent,
            pad_left=True,
            left_pad_symbol="<s>",
            pad_right=True,
            right_pad_symbol="</s>",
        )
    )
)

[('the', 'answer'), ('answer', 'is'), ('is', '42')]
[('the', 'answer', 'is'), ('answer', 'is', '42')]
[('<s>', 'the'), ('the', 'answer'), ('answer', 'is'), ('is', '42'), ('42', '</s>')]
[('<s>', 'the'), ('the', 'answer'), ('answer', 'is'), ('is', '42'), ('42', '</s>')]


#### 1.1.5. Extracting "Everygrams"

To make an N-gram model more robust, it is usually also trained on lower-order N-grams (i.e. unigrams in case of bigrams).
NLTK also provides an utility function to extract these together.

Note the `max_len` argument that defines the maximum length of an N-gram.


In [34]:
from nltk.util import everygrams

print(list(everygrams(sent, max_len=3)))
print(list(everygrams(pad_both_ends(sent, n=3), max_len=3)))

[('the',), ('the', 'answer'), ('the', 'answer', 'is'), ('answer',), ('answer', 'is'), ('answer', 'is', '42'), ('is',), ('is', '42'), ('42',)]
[('<s>',), ('<s>', '<s>'), ('<s>', '<s>', 'the'), ('<s>',), ('<s>', 'the'), ('<s>', 'the', 'answer'), ('the',), ('the', 'answer'), ('the', 'answer', 'is'), ('answer',), ('answer', 'is'), ('answer', 'is', '42'), ('is',), ('is', '42'), ('is', '42', '</s>'), ('42',), ('42', '</s>'), ('42', '</s>', '</s>'), ('</s>',), ('</s>', '</s>'), ('</s>',)]


In [35]:
all_ngrams = everygrams(
    sent,
    min_len=1,
    max_len=2,
    pad_left=True,
    left_pad_symbol="<s>",
    pad_right=True,
    right_pad_symbol="</s>",
)
list(all_ngrams)

[('<s>',),
 ('<s>', 'the'),
 ('the',),
 ('the', 'answer'),
 ('answer',),
 ('answer', 'is'),
 ('is',),
 ('is', '42'),
 ('42',),
 ('42', '</s>'),
 ('</s>',)]

#### 1.1.6. "Flattening" the Data

For language model training and evaluation (as well as vocabulary extraction) NLTK expects the data to be a flat list.
In python, this is done either by using `itertools.chain.from_iterable` or python list comprehension as

`[element for sublist in superlist for element in sublist]`


In [36]:
from itertools import chain

data = [
    ["which", "is", "the", "sense", "of", "life", "?"],
    ["the", "answer", "is", "42"],
]
list(chain.from_iterable(data))

['which', 'is', 'the', 'sense', 'of', 'life', '?', 'the', 'answer', 'is', '42']

In [37]:
# List Comprehension
[token for sent in data for token in sent]

['which', 'is', 'the', 'sense', 'of', 'life', '?', 'the', 'answer', 'is', '42']

In [38]:
# Or another trick
sum(data, [])

['which', 'is', 'the', 'sense', 'of', 'life', '?', 'the', 'answer', 'is', '42']

In [39]:
from nltk.lm.preprocessing import flatten

list(flatten(data))

['which', 'is', 'the', 'sense', 'of', 'life', '?', 'the', 'answer', 'is', '42']

#### 1.1.7. Combining the Tasks

NLTK wraps both tasks:

- padded `ngram` (everygram) extraction from each sentence
- flat list creation

into a convenient utility function `padded_everygram_pipeline` that returns lazy iterators.


In [40]:
from nltk.lm.preprocessing import padded_everygram_pipeline

padded_ngrams, flat_text = padded_everygram_pipeline(2, data)

In [41]:
for sent_ngrams in padded_ngrams:
    print(list(sent_ngrams))

[('<s>',), ('<s>', 'which'), ('which',), ('which', 'is'), ('is',), ('is', 'the'), ('the',), ('the', 'sense'), ('sense',), ('sense', 'of'), ('of',), ('of', 'life'), ('life',), ('life', '?'), ('?',), ('?', '</s>'), ('</s>',)]
[('<s>',), ('<s>', 'the'), ('the',), ('the', 'answer'), ('answer',), ('answer', 'is'), ('is',), ('is', '42'), ('42',), ('42', '</s>'), ('</s>',)]


In [42]:
list(flat_text)

['<s>',
 'which',
 'is',
 'the',
 'sense',
 'of',
 'life',
 '?',
 '</s>',
 '<s>',
 'the',
 'answer',
 'is',
 '42',
 '</s>']

**Note:** lazy iterators (aka generators) are for one-use only.


In [43]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, ["winter is coming".split()])
print(padded_ngrams)
print([list(x) for x in padded_ngrams])  # We use the generator
print([list(x) for x in padded_ngrams])  # The generator is gone
# If you want to reuse the generator you have to recompute it
padded_ngrams, flat_text = padded_everygram_pipeline(2, ["winter is coming".split()])
print([list(x) for x in padded_ngrams])

<generator object padded_everygram_pipeline.<locals>.<genexpr> at 0x78279a1fe7a0>
[[('<s>',), ('<s>', 'winter'), ('winter',), ('winter', 'is'), ('is',), ('is', 'coming'), ('coming',), ('coming', '</s>'), ('</s>',)]]
[]
[[('<s>',), ('<s>', 'winter'), ('winter',), ('winter', 'is'), ('is',), ('is', 'coming'), ('coming',), ('coming', '</s>'), ('</s>',)]]


### 1.2. Ngram Counter

NLTK provides NgramCounter class to do the ngram counting. The class can be initialized without any argument and then updated with `ngrams` (via `update()` method); or directly initialized with `ngrams`.

`N()` method returns total number of N-grams stored.


In [44]:
from nltk.lm import NgramCounter

counter = NgramCounter()
counter.N()  # return total number of stored ngrams

0

In [45]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)
counter.update(padded_ngrams)  # update counter with ngrams
counter.N()

28

In [46]:
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)
counter = NgramCounter(padded_ngrams)
counter.N()

28

#### 1.2.1. Accessing Ngram Counts

Ngram counts can be accessed using standard python dictionary notation.

- Ngram order counts can be accessed using integer keys (order)
  - returns `Frequency Distribution` or `Conditional Frequency Distribution` objects. [link](https://www.nltk.org/api/nltk.probability.html)
- Unigram counts can be accessed using string keys.
- Bigram counts can be accessed using list keys & string keys (see example)


In [47]:
# ngram order counts
print(counter[1])  # Frequency Distribution
print(counter[2])  # Conditional Frequency Distribution

<FreqDist with 11 samples and 15 outcomes>
<ConditionalFreqDist with 10 conditions>


In [48]:
# print counts
print(counter.unigrams)  # unigram count for convenience

<FreqDist with 11 samples and 15 outcomes>


In [49]:
# unigram counts
counter["the"]

2

In [50]:
# 'full' bigram counts (full bigrams)
counter[["the"]]["answer"]

1

In [51]:
# to get a frequency distribution over all continuations, you can use list or tuple.
# counter[['the']] or counter[('the',)]
counter[["the"]]

FreqDist({'sense': 1, 'answer': 1})

In [52]:
# For Conditional Frequency Distributions, only tuples are accepted.
# counter[2][['the']] with throw an error
counter[2][("the",)]

FreqDist({'sense': 1, 'answer': 1})

### Exercise 1

- Load Shakespeare's Hamlet from Gutenberg corpus

  - lowercase it

- Extract padded unigrams and bigrams

- Using NgramCounter
  - get total number of ngrams
  - get count of unigram `the`
  - get count of bigram `of the`

|                 | Count |
| --------------- | ----- |
| Ngrams          | 84038 |
| Unigram _the_   | 993   |
| Bigram _of the_ | 59    |


In [53]:
from nltk.corpus import gutenberg
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
import nltk
from nltk.lm import NgramCounter


gutenberg_plain = [[w.lower() for w in sent] for sent in gutenberg.sents("shakespeare-hamlet.txt") ]
ngrams, text = padded_everygram_pipeline(2, gutenberg_plain)
ngram_counter = NgramCounter(ngrams)

print("Total number of ngrams: ", ngram_counter.N())
print("Count of unigram 'the': ", ngram_counter["the"])
print("Count of bigram 'of the': ", ngram_counter[["of"]]["the"])

# double check

Total number of ngrams:  84038
Count of unigram 'the':  993
Count of bigram 'of the':  59


In [54]:
# padded_ngrams, flat_text = padded_everygram_pipeline(2, hamlet_lowercase)

In [55]:
counter = NgramCounter(padded_ngrams)

In [56]:
print(counter.N())  # Total number of Ngrams
print(counter["the"])  # the
print(counter[["of"]]["the"])  # of the

0
0
0


## 2. Vocabulary and Basic Usage


[`Vocabulary`](https://www.nltk.org/api/nltk.lm.vocabulary.html) class of NLTK satisfies two common language modeling requirements for a vocabulary:

- When checking membership and calculating its size, filters items by comparing their counts to a cutoff value.
- Adds a special "unknown" token which unseen words are mapped to.

Tokens with counts greater than or equal to the cut-off value will be considered part of the vocabulary.


`Vocabulary(counts=None, unk_cutoff=1, unk_label='<UNK>)` create a new `Vocabulary`.

- `counts` (optional) iterable or collections.

  - Counter instance to pre-seed the `Vocabulary`.
  - In case it is iterable, counts are calculated.

- `unk_cutoff` (int) - Words that occur less frequently than this value are not considered part of the vocabulary.

- `unk_label` - Label for marking words not part of vocabulary.


In [57]:
from nltk.lm import Vocabulary
from nltk.corpus import gutenberg

hamlet_words = gutenberg.words("shakespeare-hamlet.txt")

# lowercase
hamlet_words = [w.lower() for w in hamlet_words]

# initialize vocabulary with cut-off
vocab = Vocabulary(hamlet_words, unk_cutoff=2)

`Vocabulary` methods:

- `lookup()` looks up words in a vocabulary.
  - "Unseen" words (with counts less than cutoff) are looked up as the unknown label.
  - If given one word (a string) as an input, this method will return a string.
  - If given a sequence, it will return an tuple of the looked up words.
- `update()` update counts from a sequence


`Vocabulary` properties:

- `cutoff` - same as `unk_cutoff`

- `unk_label` and `counts` can also be accessed as properties
  - `vocab.unk_label`
  - `vocab.counts`


### 2.1. Counts, Vocabulary Membership, and Lookup

Tokens with frequency counts less than the `cutoff` value will be considered not part of the vocabulary, even though their entries in the count dictionary are preserved.

This is useful for changing cut-off without recomputing counts.


In [58]:
# let's define a function to illustrate the relation between counts, membership, and lookup
def test_word(word, vocab):
    # let's lowercase it
    if word != vocab.unk_label:
        word = word.lower()
    # membership test
    word_membership = word in vocab
    # accessing count
    word_count = vocab[word]
    # lookup
    word_lookup = vocab.lookup(word)

    print(
        "Word: '{}', Count: {}, In Vocab: {}, Mapped to: '{}'".format(
            word, word_count, word_membership, word_lookup
        )
    )

In [59]:
for w in ["<UNK>", "the", "1599", "Trento"]:
    test_word(w, vocab)

Word: '<UNK>', Count: 2, In Vocab: True, Mapped to: '<UNK>'
Word: 'the', Count: 993, In Vocab: True, Mapped to: 'the'
Word: '1599', Count: 1, In Vocab: False, Mapped to: '<UNK>'
Word: 'trento', Count: 0, In Vocab: False, Mapped to: '<UNK>'


### 2.2. Cut-Off and Vocabulary Size

The cut-off value influences not only membership checking but also the result of getting the size of the vocabulary using the built-in `len`. Note that while the number of keys in the vocabulary's counter remains the same, the items in the vocabulary differ depending on the cut-off.


In [60]:
# Let's define another vocabulary with cut-off 1
# Notice that we are passing counts from the original vocabulary
vocab1 = Vocabulary(vocab.counts, unk_cutoff=1)

In [61]:
print("CutOff 2:", len(vocab))
print("CutOff 1:", len(vocab1))

CutOff 2: 1867
CutOff 1: 4717


In [62]:
print("CutOff 2 Counts:", len(vocab.counts))
print("CutOff 1 Counts:", len(vocab1.counts))

CutOff 2 Counts: 4716
CutOff 1 Counts: 4716


#### Exercise 2

- lookup in vocabulary
  - "trento is the capital city of trentino"
- update vocabulary with "trento is the capital city of trentino"
  - do the lookup again to see the effect
- experiment with changing the cut-off value from `1` to `10`
  - do the lookup again to see the effect


In [63]:
sentence = (
    "trento is the capital city of trentino".split()
)  # use the .split() to tokenize the sentence

# Cut-off 0
print("Cut-off 0")
vocab = Vocabulary(hamlet_words, unk_cutoff=5)
# Print lookup before vocab update using vocab.lookup
for word in sentence:
    print(vocab.lookup(word))
# Update vocab using vocab.update
vocab.update(sentence)
for word in sentence:
    print(vocab.lookup(word))
# Print lookup after vocab update using vocab.lookup

Cut-off 0
<UNK>
is
the
<UNK>
<UNK>
of
<UNK>
<UNK>
is
the
<UNK>
<UNK>
of
<UNK>


In [64]:
# Cut-off 1
print("Cut-off 1")
vocab = Vocabulary(hamlet_words, unk_cutoff=1)
# Print lookup before vocab update using vocab.lookup
for word in sentence:
    print(vocab.lookup(word))
# Update vocab using vocab.update
vocab.update(sentence)
for word in sentence:
    print(vocab.lookup(word))
# Print lookup after vocab update using vocab.lookup
# Cut-off 1
print("Cut-off 10")
vocab = Vocabulary(hamlet_words, unk_cutoff=10)
# Print lookup before vocab update using vocab.lookup
for word in sentence:
    print(vocab.lookup(word))
# Update vocab using vocab.update
vocab.update(sentence)
for word in sentence:
    print(vocab.lookup(word))
# Print lookup after vocab update using vocab.lookup

Cut-off 1
<UNK>
is
the
<UNK>
city
of
<UNK>
trento
is
the
capital
city
of
trentino
Cut-off 10
<UNK>
is
the
<UNK>
<UNK>
of
<UNK>
<UNK>
is
the
<UNK>
<UNK>
of
<UNK>


## 3. Training Language Model


Using the data prepared as we have seen above, the `NLTK` ngram model boils down to counting ngrams, as presented above, by specifying the highest ngram size order.


In [65]:
# Let's prepare data on hamlet
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.corpus import gutenberg

data = [[w.lower() for w in sent] for sent in gutenberg.sents("shakespeare-hamlet.txt")]
padded_ngrams, flat_text = padded_everygram_pipeline(2, data)

In [66]:
# Let's train a `Maximum Likelihood Estimator (MLE)
from nltk.lm import MLE

mle_lm = MLE(2)  # Where 2 is the highest N-gram size order

Initialization of the Language Model creates:

- empty vocabulary
- empty counts

These are populated once we `fit` the model with training data.


In [67]:
# .vocab refers to the Vocabulary class that we have seen before
print(mle_lm.vocab)
# .vocab refers to the NgramCounter class that we have seen before
print(mle_lm.counts)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 0 items>
<NgramCounter with 1 ngram orders and 0 ngrams>


In [68]:
mle_lm.fit(padded_ngrams, flat_text)

In [69]:
print(mle_lm.vocab)
print(mle_lm.counts)
generated_text = "<s>"
for i in range(500):
    generated_text += mle_lm.generate(text_seed=generated_text) + " "
print(generated_text)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 4719 items>
<NgramCounter with 2 ngram orders and 84038 ngrams>
<s>salt , as <s> ophelia </s> . , that and and awhile ends thoughts . <s> , christian , was ' i tyrannically more , be </s> foule passion : ' say <s> - our ; lordship palme friends is good hamlet <s> giuing but masters . <s> </s> king </s> ; the . <s> the of heere damon sweet might drinke and </s> which multitude mother s to , most told tristfull and hent speake neede ' ' , re o boughes , contracted his but , , maiestie </s> <s> <s> </s> you appurtenance </s> </s> , there </s> or then is thou nor so e no report do the is iulius as holding life <s> may </s> for come thee ' , </s> , barn my </s> though giue </s> beast you beguile . health ? </s> qu our ; eare , is </s> to , of father ' : appeares in vngalled </s> shooes in honour faire </s> shuffling from it . play made if <s> not my sixe of giue prince ham his , yours the in phrase be your . foe on haue our ' do </s> honor <s>

## 4. Using Ngram Language Models

A statistical [language model](https://en.wikipedia.org/wiki/Language_model) is a probability distribution over sequences of words. Given such a sequence, say of length $n$, it assigns a probability $P(w_{1},\ldots ,w_{n})$ ($P(w_{1}^{n})$, for compactness) to the whole sequence (using Chain Rule). Consequently, the unigram and bigram probabilities computed above constitute an ngram language model of our corpus.

<!-- It is more useful for Natural Language Processing to have a __probability__ of a sequence being legal, rather than a grammar's __boolean__ decision whether it is legal or not. -->


### 4.1. Computing Probability of a Sequence (Scoring)

The most common usage of a language model is to compute probability of a sequence.


#### Probability of a Sequence

Probability of a sequence is computed as a product of conditional probabilities ([Chain Rule](<https://en.wikipedia.org/wiki/Chain_rule_(probability)>)).

$$P(w_{1}^{n}) = P(w_1) P(w_2|w_1) P(w_3|w_1^2) ... P(w_n|w_{1}^{n-1}) = \prod_{i=1}^{n}{P(w_i|w_{1}^{i-1})}$$

The order of ngram makes a simplifying assumption that probability of a current word only depends on previous $N - 1$ elements. Thus, it truncates previous context (history) to length $N - 1$.

$$P(w_i|w_{1}^{i-1}) \approx P(w_i|w_{i-N+1}^{i-1})$$

Consequently we have:
\begin{align*}
Unigram:\;& P(w*i) \\
Bigram: \;& P(w_i|w*{i-1}) \\
Trigram: \;& P(w*i|w*{i-2},w\_{i-1}) \\
\end{align*}

<!--| N-gram   | Equation                     |
:--------:|:---------------------------:|
unigram  | $P(w_i)$                     |
bigram   | $P(w_i|w_{i-1})$             |
trigram  | $P(w_i|w_{i-2},w_{i-1})$     |
-->

The probability of the whole sequence applying an ngram model becomes:

$$P(w_{1}^{n}) = \prod_{i=1}^{n}{P(w_i|w_{i-N+1}^{i-1})}$$


#### Calculating Probability from Frequencies

Probabilities of ngrams can be computed by _normalizing_ frequency counts (aka _Maximum Likelihood Estimation_): dividing the frequency of an ngram sequence by the frequency of its prefix (_relative frequency_).
\begin{align*}
Unigram:\; & P(w*i) = \frac{c(w_i)}{N}\\
Bigram:\; & P(w_i | w*{i-1}) = \frac{c(w*{i-1}, w_i)}{c(w*{i-1})}\\
Ngram: \; & P(w*i | w*{i-n+1}^{i-1}) = \frac{c(w*{i-n+1}^{i-1}, w_i)}{c(w*{i-n+1}^{i-1})}\\
\end{align*}

<!--
N-gram   | Equation
:--------|:------------------------------
Unigram  | $$p(w_i) = \frac{c(w_i)}{N}$$
Bigram   | $$p(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_{i-1})}$$
Ngram    | $$p(w_i | w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^{i-1}, w_i)}{c(w_{i-n+1}^{i-1})}$$  -->

where:

- $N$ is the total number of words in a corpus
- $c(x)$ is the count of occurrences of $x$ in a corpus (x could be unigram, bigram, etc.)


#### 4.1.1. Scoring Methods of NLTK LMs

- `score(word, context=None)` masks out of vocab (OOV) words (i.e. maps them to `<UNK>`) and computes their model score.
  - scores are **model-specific** (later on this)
  - Parameters:
    - `word (str)` - Word for which we want the score
    - `context (tuple(str))` - Context the word is in. If None, computes unigram score.
- `logscore(word, context=None)` - Evaluate the log score of this word in this context.


In [70]:
# A little illustration of the methods
mle_lm.score("the")

0.02278986505095015

In [71]:
mle_lm.score("the", ["of"])

0.09672131147540984

In [72]:
import math

mle_lm.logscore(
    "the", ["of"]
)  # == math.log(mle_lm.logscore("the", ["of"]), 2) log base 2

-3.3700223830884077

#### Exercise 3

Implement a function to compute score of a sequence (i.e. Chain Rule)

- arguments:

  - Language Model
  - List of Tokens

- functionality

  - extracts ngrams w.r.t. LM order (`lm.order`)
  - scores each ngram w.r.t. LM (`lm.score` or `lm.logscore`)
    - note: `score` already takes care of OOV by converting to `<UNK>`
  - computes the overall score using chain rule
    - note: the difference between `score` and `logscore`

- compute the scores of the `test_sents`
  - compute the scores of padded and unpadded sequences


In [73]:
# Toy test set
test_sents = ["the king is dead", "the tzar is dead", "the tragedie of hamlet is good"]

In [74]:
def chain_rule(lm, sentence, log=True, pad=True):
    ngrams = everygrams(
        sentence.split(" "),
        min_len=1,
        max_len=lm.order,
        pad_left=pad,
        left_pad_symbol="<s>",
        pad_right=pad,
        right_pad_symbol="</s>",
    )

    ngrams = flatten(ngrams)

    total_score = int(not log)
    for ngram in ngrams:
        if log:
            word_score = lm.logscore(ngram)
            total_score += word_score
        else:
            word_score = lm.score(ngram)
            total_score *= word_score

    return total_score


for sent in test_sents:
    print(sent, chain_rule(mle_lm, sent, log=True, pad=True))

the king is dead -108.23537920342142
the tzar is dead -inf
the tragedie of hamlet is good -164.16783314846086


#### 4.1.2. OOV in MLE

In MLE LM we did not take care of OOV. Consequently, those have `0` counts and probabilities.


In [75]:
print(mle_lm.score("<UNK>"))
print(mle_lm.score("<UNK>", ["<UNK>"]))
print(mle_lm.score("<UNK>", ["the"]))

0.0
0
0.0


In [76]:
# same as above: getting lowercaseed sentences and words
hamlet_sents = [
    [w.lower() for w in sent] for sent in gutenberg.sents("shakespeare-hamlet.txt")
]
hamlet_words = flatten(hamlet_sents)

In [77]:
# computing vocabulary with cutoff
lex = Vocabulary(hamlet_words, unk_cutoff=2)
print(len(lex))

1867


In [78]:
# replacing words with counts below cutoff with '<UNK>'
hamlet_oov_sents = [list(lex.lookup(sent)) for sent in hamlet_sents]
print(hamlet_oov_sents[0])

['[', 'the', 'tragedie', 'of', 'hamlet', 'by', '<UNK>', '<UNK>', '<UNK>', ']']


In [79]:
# extracting ngrams & words again
padded_ngrams_oov, flat_text_oov = padded_everygram_pipeline(2, hamlet_oov_sents)

In [80]:
lm_oov = MLE(2)
lm_oov.fit(padded_ngrams_oov, flat_text_oov)

In [81]:
print(lm_oov.score("<UNK>"))
print(lm_oov.score("<UNK>", ["<UNK>"]))
print(lm_oov.score("<UNK>", ["the"]))

0.06540897824290828
0.06842105263157895
0.24874118831822759


#### 4.1.3. Data Sparsity: Missing Ngrams

However, even though we have counted ngrams on the data set with `<UNK>`, the counts still do not account for all possible ngrams. Thus, some possible sequences will have zero probability.

**We will address this later.**


In [82]:
print(lm_oov.score("queen"))
print(lm_oov.score("<UNK>", ["queen"]))
print(lm_oov.score("queen", ["<UNK>"]))

0.00018360414945377767
0.0
0.0


### 4.2. Generating Sequences

Ngram Model can be used as an automaton to generate probable legal sequences using the algorithm below.

**Algorithm for Bigram LM**

- $w\_{i-1} = $ `<s>`;
- _while_ $w_i \neq $ `</s>`

  - stochastically get new word w.r.t. $P(w_i|w_{i-1})$


#### 4.2.1. Generation Method of NLTK LM

- `generate(num_words=1, text_seed=None, random_seed=None)` generates words from the model.
  - Parameters
    - `num_words (int)` - How many words to generate. By default 1.
    - `text_seed` - preceding context the generation should be conditioned on.
      - ngram model is restricted in how much preceding context it can take into account based on max ngram size
    - `random_seed` - A random seed. If provided, makes the random sampling part of generation reproducible.


In [83]:
mle_lm.generate(5)

["'", 'right', 'well', 'to', 'put']

In [84]:
# Note that you have to manually truncate the sentence when there is a </s>
mle_lm.generate(5, text_seed=["<s>", "hamlet"])

['?', '</s>', 'that', 'speech', '</s>']

### 4.3. Language Model Evaluation

Language Models are evaluated using [Perplexity](https://en.wikipedia.org/wiki/Perplexity)

- Measures how well model fits test data
- Probability of test data
- Weighted average branching factor in predicting the next word (lower is better).
- Computed as:
  \begin{align*}
  PP(W) = \sqrt[N]{\frac{1}{P(w*1,w_2,...,w_N)}} = \sqrt[N]{\frac{1}{\prod*{i=1}^{N}P(w*i|w*{i-N+1})}}
  \end{align*}

(abbreviated as PPL also)

Where $N$ is the number of words in test set.


#### 4.3.1. Evaluation Methods of NLTK LM


`NLTK` provides both **perplexity** and **cross-entropy** as evaluation methods

- `entropy(text_ngrams)` calculates cross-entropy of model for given evaluation text.

  - Parameters
    - `text_ngrams (Iterable(tuple(str)))` A sequence of ngram tuples.

- `perplexity(text_ngrams)` calculates the perplexity of the given text.
  - This is 2<sup>cross-entropy</sup> for the text, so the arguments are the same.


**Perplexity** is related to [**Cross-Entropy**](https://en.wikipedia.org/wiki/Cross_entropy) as:
\begin{align*}
PP(p) = 2^{H(p)}
\end{align*}


#### Exercise 4

Compute entropy and perplexity of the `MLE` models on the bigrams of the test sentences below, treating them as a test set.

- experiment with the two test sets
- experiment with OOVs (with vs without)


In [85]:
test_sents1 = ["i saw him yesternight", "he was a man"]
test_sents2 = ["the king is dead", "welcome to you", "how are you"]

In [96]:
# Load data
test_sents1_tokenized = [sent.split() for sent in test_sents1]
test_sents2_tokenized = [sent.split() for sent in test_sents2]

ngrams1, _ = padded_everygram_pipeline(order=2, text=test_sents1_tokenized)
ngrams2, _ = padded_everygram_pipeline(order=2, text=test_sents2_tokenized)

ngram1_list = [list(x) for x in ngrams1]
ngram2_list = [list(x) for x in ngrams2]
print("Cross entropy and perplexity of test set 1 with oov and without oov")

for sent in ngram1_list:
    h_1 = lm_oov.entropy(sent)
    h_2 = mle_lm.entropy(sent)
    print("Entropy: ", h_1, h_2, "Perplexity: ", 2**h_1, 2**h_2)  # Fixed perplexity calculation

print("Cross entropy and perplexity of test set 2 with oov and without oov")

for sent in ngram2_list:
    h_1 = lm_oov.entropy(sent)
    h_2 = mle_lm.entropy(sent)

    print("Entropy: ", h_1, h_2, "Perplexity: ", 2**h_1, 2**h_2)  # Fixed perplexity calculation


Cross entropy and perplexity of test set 1 with oov and without oov
Entropy:  5.366023181550911 6.21829949349154 Perplexity:  41.24145084717631 74.45513742535586
Entropy:  5.941056297216614 5.941056297216614 Perplexity:  61.437870104068715 61.437870104068715
Cross entropy and perplexity of test set 2 with oov and without oov
Entropy:  5.873388558622813 5.873388558622813 Perplexity:  58.62274256918178 58.62274256918178
Entropy:  6.164160726635512 6.164160726635512 Perplexity:  71.7128985976673 71.7128985976673
Entropy:  6.038466889604594 6.038466889604594 Perplexity:  65.72939904377688 65.72939904377688


In [97]:
# Test your specific bigram with proper padding
test_sent = test_sents1[1]
test_set = [test_sent]

# Create padded n-grams properly using your function
padded_test_ngrams, padded_test_text = padded_everygram_pipeline(
    lm_oov.order, [lex.lookup(sent.split()) for sent in test_set]
)

# Convert the generator to a list for inspection
all_test_ngrams = list(padded_test_ngrams)
flattened_test_ngrams = [item for sublist in all_test_ngrams for item in sublist]

# Look at the actual bigrams created
bigrams_only = [ng for ng in flattened_test_ngrams if len(ng) == 2]
print("Padded bigrams created by padded_everygram_pipeline:")
for bigram in bigrams_only:
    score = lm_oov.score(bigram[-1], bigram[:-1])
    print(f"P({bigram}) = {score}")

Padded bigrams created by padded_everygram_pipeline:
P(('<s>', 'he')) = 0.008048937540244688
P(('he', 'was')) = 0.0297029702970297
P(('was', 'a')) = 0.09523809523809523
P(('a', 'man')) = 0.025440313111545987
P(('man', '</s>')) = 0.06521739130434782


##### PPL: how it works inside


In [98]:
import math
import numpy as np


def compute_ppl(model, data):
    highest_ngram = model.order
    scores = []
    for sentence in data:
        ngrams, _ = padded_everygram_pipeline(highest_ngram, [sentence.split()])
        scores.extend(
            [
                -1.0 * model.logscore(w[-1], w[0:-1])
                for gen in ngrams
                for w in gen
                if len(w) == highest_ngram
            ]
        )
    return math.pow(2.0, np.asarray(scores).mean())


compute_ppl(mle_lm, test_sents2)

49.091034440830974

### 4.4. Handling Data Sparseness: Smoothing in NLTK


[`Smoothing(vocabulary, counter, **kwargs)`](https://www.nltk.org/api/nltk.lm.smoothing.html) Class is initialized with the following parameters:

- `vocabulary (nltk.lm.vocab.Vocabulary)` - The Ngram vocabulary object.
- `counter (nltk.lm.counter.NgramCounter)` - The counts of the vocabulary items.


`Ngram Smoothing Interface` implements [Chen & Goodman 1995](https://aclanthology.org/P96-1041.pdf)'s idea that all smoothing algorithms have certain features in common. Consequently, each Smoothing subclass implements two methods:

- `unigram_score(word)` return unigram score
- `alpha_gamma(word, context)` returns alpha and gamma values (varies w.r.t. method)
  - `gamma` is the value added to missing ngram count
  - `alpha` is the value used to scale the lower order probabilities


The following smoothing methods are implemented:

- `WittenBell(vocabulary, counter, **kwargs)` - Witten-Bell smoothing

- `AbsoluteDiscounting(vocabulary, counter, discount=0.75, **kwargs)` - Smoothing with absolute discount
  - takes `discount=0.75` parameter (default 0.75)
- `KneserNey(vocabulary, counter, order, discount=0.1, **kwargs)` - Kneser-Ney Smoothing (an extension of smoothing with a discount)


### 4.5. NLTK Language Models

All the language models in NLTK share the same interface and share methods for

- evaluation
- generation
- scoring


**Scoring** in a language model does the following:

- takes care of OOV words (see above)
- computes `unmasked_score`

The way `unmasked_score` is computed from counts is the difference.


Some language models use **smoothing** to account for missing ngrams, some do not (e.g. `MLE`)


Available LM Implementations

- `MLE` - providing MLE ngram model scores.

- Additive Smoothing LMs

  - `Laplace` - Laplace (add one) smoothing ($\gamma = 1$)
  - `Lidstone` - same as `Laplace`, but adds $\gamma$

- Back-Off Language Models

  - `StupidBackoff` - uses `alpha` to scale the lower order probabilities for computing missing ngram scores

- Interpolated Language Models
  - `WittenBellInterpolated` - Interpolated version of Witten-Bell smoothing
  - `KneserNeyInterpolated` - Interpolated version of Kneser-Ney smoothing.
  - `AbsoluteDiscountingInterpolated` - Interpolated version of smoothing with absolute discount


## Last Exercise: Language Model Evaluation

Write your own implementation of the Stupid backoff algorithm. Train it and compare the perplexity with the one provided by NLKT. The dataset that you have to use is the _Shakespeare Macbeth_. You have to split the dataset in training, development and test sets. The train the model on the training set, find the best ⍺ on the dev set, and test the model on the test set.

Stupid Backoff algorithm (use ⍺=0.4):
https://aclanthology.org/D07-1090.pdf

NLTK (StupidBackoff):
https://www.nltk.org/api/nltk.lm.html

**Suggestion**: adapt the `compute_ppl` function to compute the perplexity of your model. The PPL has to be computed on the whole corpus and not at sentence level.


In [ ]:
from typing import Any


class StupidBackoff:
    def __init__(self, max_size, gamma):
        self.max_size = max_size
        self.gamma = gamma
        self.mle_models = [MLE(i) for i in range(max_size)]

    def fit(self, text: Any, vocabulary_text: Any | None = None):
        for mle_model in self.mle_models:
            mle_model.fit(text, vocabulary_text)

    def logscore(self, random_variable, conditioned_variable):
        highest_order_score = self.mle_models[-1].logscore(random_variable, conditioned_variable)
        if highest_order_score == 0:
            model_score = self.max_size - 2
            lower_order_score = self.mle_models[model_score].logscore(random_variable, conditioned_variable)
            while lower_order_score == 0:
                model_score -=1
                lower_order_score = self.mle_models[model_score].logscore(random_variable, conditioned_variable)
        else:
            return highest_order_score * self.gamma

In [ ]:
# NLTK StupidBackoff
from nltk.lm import StupidBackoff
import random
# Dataset
macbeth_sents = [
    [w.lower() for w in sent] for sent in gutenberg.sents("shakespeare-macbeth.txt")
]

sents_len = len(macbeth_sents)
test_train_split = int(sents_len * 0.80)
random.shuffle(macbeth_sents)

train = macbeth_sents[:test_train_split]
test = macbeth_sents[test_train_split:]
print(test)
# Split the corpus 'shakespeare-macbeth.txt' into train, dev, test as we have seen in LAB 02

from sklearn.model_selection import KFold
n_split = 5
random_split = KFold(n_splits=n_split, shuffle=True)
for training, val in random_split.split(train):
    # print("Training: ", training, "Val: ", val)

[['what', 'newes', '?'], ['la', '.'], ['sey', '.'], ['was', 'he', 'not', 'borne', 'of', 'woman', '?'], ['with', 'this', 'strange', 'vertue', ',', 'he', 'hath', 'a', 'heauenly', 'guift', 'of', 'prophesie', ',', 'and', 'sundry', 'blessings', 'hang', 'about', 'his', 'throne', ',', 'that', 'speake', 'him', 'full', 'of', 'grace', '.'], ['torches', '.'], ['wife', '.'], ['looke', 'how', 'she', 'rubbes', 'her', 'hands'], ['rosse', '.'], ['2', 'fillet', 'of', 'a', 'fenny', 'snake', ',', 'in', 'the', 'cauldron', 'boyle', 'and', 'bake', ':', 'eye', 'of', 'newt', ',', 'and', 'toe', 'of', 'frogge', ',', 'wooll', 'of', 'bat', ',', 'and', 'tongue', 'of', 'dogge', ':', 'adders', 'forke', ',', 'and', 'blinde', '-', 'wormes', 'sting', ',', 'lizards', 'legge', ',', 'and', 'howlets', 'wing', ':', 'for', 'a', 'charme', 'of', 'powrefull', 'trouble', ',', 'like', 'a', 'hell', '-', 'broth', ',', 'boyle', 'and', 'bubble'], ['the', 'king', '-', 'becoming', 'graces', ',', 'as', 'iustice', ',', 'verity', ',', 'te